In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_val_score # to split the data
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, fbeta_score #To evaluate our model
from sklearn import preprocessing
# from sklearn.metrics import accuracy_score

# Algorithmns models to be compared
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier

In [6]:
org_df = pd.read_csv('german_credit_data.csv')

In [3]:
org_df.shape

(1000, 11)

In [4]:
org_df

,Unnamed: 0,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,2,49,male,1,own,little,NaN,2096,12,education,good
3,3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,4,53,male,2,free,little,little,4870,24,car,bad
...,...,...,...,...,...,...,...,...,...,...,...
995,995,31,female,1,own,little,NaN,1736,12,furniture/equipment,good
996,996,40,male,3,own,little,little,3857,30,car,good
997,997,38,male,2,own,little,NaN,804,12,radio/TV,good
998,998,23,male,2,free,little,little,1845,45,radio/TV,bad


In [7]:
df1 = org_df.drop(['Unnamed: 0'], axis=1)

In [6]:
print(df1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Age               1000 non-null   int64 
 1   Sex               1000 non-null   object
 2   Job               1000 non-null   int64 
 3   Housing           1000 non-null   object
 4   Saving accounts   817 non-null    object
 5   Checking account  606 non-null    object
 6   Credit amount     1000 non-null   int64 
 7   Duration          1000 non-null   int64 
 8   Purpose           1000 non-null   object
 9   Risk              1000 non-null   object
dtypes: int64(4), object(6)
memory usage: 78.2+ KB
None


In [13]:
df1.isna().sum()

Age                   0
Sex                   0
Job                   0
Housing               0
Saving accounts     183
Checking account    394
Credit amount         0
Duration              0
Purpose               0
Risk                  0
dtype: int64

In [8]:
df1['Saving accounts'] = df1['Saving accounts'].fillna('no_inf')
df1['Checking account'] = df1['Checking account'].fillna('no_inf')

In [9]:
df1.isna().sum()

Age                 0
Sex                 0
Job                 0
Housing             0
Saving accounts     0
Checking account    0
Credit amount       0
Duration            0
Purpose             0
Risk                0
dtype: int64

In [117]:
sex = {}
counter = 0
for each in sorted(df1['Sex'].unique()): 
    sex[each] = counter 
    counter = counter + 1

In [118]:
housing = {}
counter = 0
for each in sorted(df1['Housing'].unique()): 
    housing[each] = counter 
    counter = counter + 1

In [119]:
sa = {}
counter = 0
for each in sorted(df1['Saving accounts'].unique()): 
    sa[each] = counter 
    counter = counter + 1

In [120]:
ca = {}
counter = 0
for each in sorted(df1['Checking account'].unique()): 
    ca[each] = counter 
    counter = counter + 1

In [121]:
purp = {}
counter = 0
for each in sorted(df1['Purpose'].unique()): 
    purp[each] = counter 
    counter = counter + 1

In [127]:
risk = {}
counter = 0
for each in sorted(df1['Risk'].unique()): 
    risk[counter] = each 
    counter = counter + 1

In [123]:
job = {
    "unskilled and non-resident" : 0, 
    "unskilled and resident" : 1, 
    "skilled" : 2, 
    "highly skilled": 3
}

In [89]:
list(job.keys())

['unskilled and non-resident',
 'unskilled and resident',
 'skilled',
 'highly skilled']

In [128]:
risk

{0: 'bad', 1: 'good'}

In [129]:
json_obj = {
    "sex": sex, 
    "Housing": housing, 
    "Saving_accounts": sa,
    "Checking_account": ca,
    "Purpose": purp,
    "Risk": risk, 
    "job": job, 
}

In [130]:
json_obj

{'sex': {'female': 0, 'male': 1},
 'Housing': {'free': 0, 'own': 1, 'rent': 2},
 'Saving_accounts': {'little': 0,
  'moderate': 1,
  'no_inf': 2,
  'quite rich': 3,
  'rich': 4},
 'Checking_account': {'little': 0, 'moderate': 1, 'no_inf': 2, 'rich': 3},
 'Purpose': {'business': 0,
  'car': 1,
  'domestic appliances': 2,
  'education': 3,
  'furniture/equipment': 4,
  'radio/TV': 5,
  'repairs': 6,
  'vacation/others': 7},
 'Risk': {0: 'bad', 1: 'good'},
 'job': {'unskilled and non-resident': 0,
  'unskilled and resident': 1,
  'skilled': 2,
  'highly skilled': 3}}

In [115]:
import json

In [131]:
with open('factor_codes.json', 'w') as file:
    json.dump(json_obj, file)

In [101]:
factors = {"factors": 
 [
     {
        "factor_name": "Sex", 
        "range" : list(df1['Sex'].unique()),   
        "interval": None, 
        "type": "categorical"
    },
    {
        "factor_name": "Housing", 
        "range" : list(df1['Housing'].unique()), 
        "interval": None, 
        "type": "categorical"
    },
    {
        "factor_name": "Saving accounts", 
        "range" : list(df1['Saving accounts'].unique()),   
        "interval": None, 
        "type": "categorical"
    },
    {
        "factor_name": "Checking account", 
        "range" : list(df1['Checking account'].unique()),   
        "interval": None, 
        "type": "categorical"
    },
    {
        "factor_name": "Purpose", 
        "range" : list(df1['Purpose'].unique()),   
        "interval": None, 
        "type": "categorical"
    },
    {
        "factor_name": "Job", 
        "range" : list(job.keys()),   
        "interval": None, 
        "type": "categorical"
    },
    {
        "factor_name": "Credit amount", 
        "range" : [df1['Credit amount'].min(), df1['Credit amount'].max()],   
        "interval": 0.5, 
        "type": "continous"
    }, 
    {
        "factor_name": "Age", 
        "range" : [df1['Age'].min(), df1['Age'].max()],   
        "interval": 1, 
        "type": "categorical"
    }, 
    {
        "factor_name": "Duration in Months", 
        "range" : [df1['Duration'].min(), df1['Duration'].max()],   
        "interval": 1, 
        "type": "categorical"
    }
 ]
}

In [103]:
with open("factors.json", 'w') as file: 
    json.dump(factors, file)

In [16]:
df1['Sex'] = df1['Sex'].astype('category')
df1['Housing'] = df1['Housing'].astype('category')
df1['Saving accounts'] = df1['Saving accounts'].astype('category')
df1['Checking account'] = df1['Checking account'].astype('category')
df1['Purpose'] = df1['Purpose'].astype('category')
df1['Risk'] = df1['Risk'].astype('category')

df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   Age               1000 non-null   int64   
 1   Sex               1000 non-null   category
 2   Job               1000 non-null   int64   
 3   Housing           1000 non-null   category
 4   Saving accounts   1000 non-null   category
 5   Checking account  1000 non-null   category
 6   Credit amount     1000 non-null   int64   
 7   Duration          1000 non-null   int64   
 8   Purpose           1000 non-null   category
 9   Risk              1000 non-null   category
dtypes: category(6), int64(4)
memory usage: 38.3 KB


In [17]:
df1['Sex'] = df1['Sex'].cat.codes
df1['Housing'] = df1['Housing'].cat.codes
df1['Saving accounts'] = df1['Saving accounts'].cat.codes
df1['Checking account'] = df1['Checking account'].cat.codes
df1['Purpose'] = df1['Purpose'].cat.codes
df1['Risk'] = df1['Risk'].cat.codes

In [18]:
df1

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,1,2,1,2,0,1169,6,5,1
1,22,0,2,1,0,1,5951,48,5,0
2,49,1,1,1,0,2,2096,12,3,1
3,45,1,2,0,0,0,7882,42,4,1
4,53,1,2,0,0,0,4870,24,1,0
...,...,...,...,...,...,...,...,...,...,...
995,31,0,1,1,0,2,1736,12,4,1
996,40,1,3,1,0,0,3857,30,1,1
997,38,1,2,1,0,2,804,12,5,1
998,23,1,2,0,0,0,1845,45,5,0


In [21]:
x = df1.drop('Risk', 1).values

y = df1['Risk'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state=42)

In [22]:
#Bagging Classifier
bag = BaggingClassifier(n_estimators=2)
fitted_bag = bag.fit(x, y)
y_pred = bag.predict(x)
accuracy_score(y,y_pred)

0.861

In [56]:
import pickle

with open('credit_risk_trained_model.model','wb') as f:
    pickle.dump(fitted_bag, f)

In [2]:
import pickle
model = pickle.load(open('credit_risk_trained_model.model', 'rb'))

In [39]:
for each in x:
    for ech_1 in each:
        print(ech_1)
    break

67
1
2
1
2
0
1169
6
5


In [29]:
import numpy as np

In [104]:
model.predict_proba([[67, 1, 2, 1, 2, 0, 1169, 6, 5]])

array([[0., 1.]])

In [132]:
model.predict([[22, 0, 2, 1, 0, 1, 5951, 48, 5]])

array([0], dtype=int8)

In [19]:
df1.shape

(1000, 10)

In [20]:
df1.columns

Index(['Age', 'Sex', 'Job', 'Housing', 'Saving accounts', 'Checking account',
       'Credit amount', 'Duration', 'Purpose', 'Risk'],
      dtype='object')

In [11]:
df1.describe()

,Age,Job,Credit amount,Duration
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,35.546000,1.904000,3271.258000,20.903000
std,11.375469,0.653614,2822.736876,12.058814
min,19.000000,0.000000,250.000000,4.000000
25%,27.000000,2.000000,1365.500000,12.000000
50%,33.000000,2.000000,2319.500000,18.000000
75%,42.000000,2.000000,3972.250000,24.000000
max,75.000000,3.000000,18424.000000,72.000000


In [12]:
df1.corr()

,Age,Job,Credit amount,Duration
Age,1.000000,0.015673,0.032716,-0.036136
Job,0.015673,1.000000,0.285385,0.210910
Credit amount,0.032716,0.285385,1.000000,0.624984
Duration,-0.036136,0.210910,0.624984,1.000000


In [13]:
df1.nunique()

Age                  53
Sex                   2
Job                   4
Housing               3
Saving accounts       4
Checking account      3
Credit amount       921
Duration             33
Purpose               8
Risk                  2
dtype: int64

In [14]:
df_good = df1.loc[df1["Risk"] == 'good']['Age'].values.tolist()
df_bad = df1.loc[df1["Risk"] == 'bad']['Age'].values.tolist()
df_age = df1['Age'].values.tolist()

In [15]:
interval = (18, 25, 35, 60, 120)

cats = ['Student', 'Young', 'Adult', 'Senior']
df1["Age_cat"] = pd.cut(df1.Age, interval, labels=cats)


df_good = df1[df1["Risk"] == 'good']
df_bad = df1[df1["Risk"] == 'bad']

In [16]:
def one_hot_encoder(df1, nan_as_category = False):
    original_columns = list(df1.columns)
    categorical_columns = [col for col in df1.columns if df1[col].dtype == 'object']
    df1 = pd.get_dummies(df1, columns= categorical_columns, dummy_na= nan_as_category, drop_first=True)
    new_columns = [c for c in df1.columns if c not in original_columns]
    return df1, new_columns

In [17]:
mod_df = one_hot_encoder(org_df)

In [18]:
org_df

,Unnamed: 0,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,2,49,male,1,own,little,NaN,2096,12,education,good
3,3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,4,53,male,2,free,little,little,4870,24,car,bad
...,...,...,...,...,...,...,...,...,...,...,...
995,995,31,female,1,own,little,NaN,1736,12,furniture/equipment,good
996,996,40,male,3,own,little,little,3857,30,car,good
997,997,38,male,2,own,little,NaN,804,12,radio/TV,good
998,998,23,male,2,free,little,little,1845,45,radio/TV,bad


In [19]:
df1['Saving accounts'] = df1['Saving accounts'].fillna('no_inf')
df1['Checking account'] = df1['Checking account'].fillna('no_inf')

#Purpose to Dummies Variable
df1 = df1.merge(pd.get_dummies(df1.Purpose, drop_first=True, prefix='Purpose'), left_index=True, right_index=True)
#Sex feature in dummies
df1 = df1.merge(pd.get_dummies(df1.Sex, drop_first=True, prefix='Sex'), left_index=True, right_index=True)
# Housing get dummies
df1 = df1.merge(pd.get_dummies(df1.Housing, drop_first=True, prefix='Housing'), left_index=True, right_index=True)
# Housing get Saving Accounts
df1 = df1.merge(pd.get_dummies(df1["Saving accounts"], drop_first=True, prefix='Savings'), left_index=True, right_index=True)
# Housing get Risk
df1 = df1.merge(pd.get_dummies(df1.Risk, prefix='Risk'), left_index=True, right_index=True)
# Housing get Checking Account
df1 = df1.merge(pd.get_dummies(df1["Checking account"], drop_first=True, prefix='Check'), left_index=True, right_index=True)
# Housing get Age categorical
df1 = df1.merge(pd.get_dummies(df1["Age_cat"], drop_first=True, prefix='Age_cat'), left_index=True, right_index=True)

In [20]:
def data_invertor(df1):
    #Purpose to Dummies Variable
    df1 = df1.merge(pd.get_dummies(df1.Purpose, drop_first=True, prefix='Purpose'), left_index=True, right_index=True)
    #Sex feature in dummies
    df1 = df1.merge(pd.get_dummies(df1.Sex, drop_first=True, prefix='Sex'), left_index=True, right_index=True)
    # Housing get dummies
    df1 = df1.merge(pd.get_dummies(df1.Housing, drop_first=True, prefix='Housing'), left_index=True, right_index=True)
    # Housing get Saving Accounts
    df1 = df1.merge(pd.get_dummies(df1["Saving accounts"], drop_first=True, prefix='Savings'), left_index=True, right_index=True)
    # Housing get Risk
    df1 = df1.merge(pd.get_dummies(df1.Risk, prefix='Risk'), left_index=True, right_index=True)
    # Housing get Checking Account
    df1 = df1.merge(pd.get_dummies(df1["Checking account"], drop_first=True, prefix='Check'), left_index=True, right_index=True)
    # Housing get Age categorical
    df1 = df1.merge(pd.get_dummies(df1["Age"], drop_first=True, prefix='Age_cat'), left_index=True, right_index=True)
    return df1

In [21]:
org_df.merge(pd.get_dummies(org_df.Purpose, drop_first=True, prefix='Purpose'), left_index=True, right_index=True)

,Unnamed: 0,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk,Purpose_car,Purpose_domestic appliances,Purpose_education,Purpose_furniture/equipment,Purpose_radio/TV,Purpose_repairs,Purpose_vacation/others
0,0,67,male,2,own,NaN,little,1169,6,radio/TV,good,0,0,0,0,1,0,0
1,1,22,female,2,own,little,moderate,5951,48,radio/TV,bad,0,0,0,0,1,0,0
2,2,49,male,1,own,little,NaN,2096,12,education,good,0,0,1,0,0,0,0
3,3,45,male,2,free,little,little,7882,42,furniture/equipment,good,0,0,0,1,0,0,0
4,4,53,male,2,free,little,little,4870,24,car,bad,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,31,female,1,own,little,NaN,1736,12,furniture/equipment,good,0,0,0,1,0,0,0
996,996,40,male,3,own,little,little,3857,30,car,good,1,0,0,0,0,0,0
997,997,38,male,2,own,little,NaN,804,12,radio/TV,good,0,0,0,0,1,0,0
998,998,23,male,2,free,little,little,1845,45,radio/TV,bad,0,0,0,0,1,0,0


In [22]:
#Excluding the missing columns
del df1["Saving accounts"]
del df1["Checking account"]
del df1["Purpose"]
del df1["Sex"]
del df1["Housing"]
del df1["Age_cat"]
del df1["Risk"]
del df1['Risk_good']

In [23]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   Age                          1000 non-null   int64
 1   Job                          1000 non-null   int64
 2   Credit amount                1000 non-null   int64
 3   Duration                     1000 non-null   int64
 4   Purpose_car                  1000 non-null   uint8
 5   Purpose_domestic appliances  1000 non-null   uint8
 6   Purpose_education            1000 non-null   uint8
 7   Purpose_furniture/equipment  1000 non-null   uint8
 8   Purpose_radio/TV             1000 non-null   uint8
 9   Purpose_repairs              1000 non-null   uint8
 10  Purpose_vacation/others      1000 non-null   uint8
 11  Sex_male                     1000 non-null   uint8
 12  Housing_own                  1000 non-null   uint8
 13  Housing_rent                 1000 non-null   uint

In [24]:
df1 = df1.sample(frac=1)

In [25]:
x = df1.drop('Risk_bad', 1).values

y = df1['Risk_bad'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state=42)

In [19]:
# normalized_x = preprocessing.normalize(x)

In [31]:
#Bagging Classifier
bag = BaggingClassifier(n_estimators=2)
fitted_bag = bag.fit(x, y)
y_pred = bag.predict(x)
accuracy_score(y,y_pred)

0.898

In [54]:
prob_scoress = fitted_bag.predict_proba(x)

In [55]:
prob_scoress

array([[0. , 1. ],
       [1. , 0. ],
       [0.5, 0.5],
       ...,
       [0. , 1. ],
       [1. , 0. ],
       [0.5, 0.5]])

In [33]:
pred_prob_df = pd.DataFrame(prob_scoress.tolist())

In [34]:
org_df['Probability of predicting 1'] = pred_prob_df[1]

In [35]:
org_df

,Unnamed: 0,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk,Probability of predicting 1
0,0,67,male,2,own,NaN,little,1169,6,radio/TV,good,0.0
1,1,22,female,2,own,little,moderate,5951,48,radio/TV,bad,0.0
2,2,49,male,1,own,little,NaN,2096,12,education,good,1.0
3,3,45,male,2,free,little,little,7882,42,furniture/equipment,good,0.0
4,4,53,male,2,free,little,little,4870,24,car,bad,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,31,female,1,own,little,NaN,1736,12,furniture/equipment,good,0.0
996,996,40,male,3,own,little,little,3857,30,car,good,1.0
997,997,38,male,2,own,little,NaN,804,12,radio/TV,good,1.0
998,998,23,male,2,free,little,little,1845,45,radio/TV,bad,0.0


In [40]:
x_test

array([[  34,    2, 6527, ...,    1,    0,    0],
       [  23,    1, 3234, ...,    0,    0,    0],
       [  27,    1, 2708, ...,    1,    0,    0],
       ...,
       [  37,    2, 1225, ...,    0,    1,    0],
       [  24,    2, 1207, ...,    0,    0,    0],
       [  25,    2, 3509, ...,    0,    0,    0]])

In [41]:
for each in x:
    print(x)

[[  52    1  362 ...    0    1    0]
 [  30    2 3069 ...    1    0    0]
 [  34    3 6850 ...    1    0    0]
 ...
 [  68    2 6761 ...    0    0    1]
 [  28    3 2606 ...    1    0    0]
 [  29    3 2679 ...    1    0    0]]
[[  52    1  362 ...    0    1    0]
 [  30    2 3069 ...    1    0    0]
 [  34    3 6850 ...    1    0    0]
 ...
 [  68    2 6761 ...    0    0    1]
 [  28    3 2606 ...    1    0    0]
 [  29    3 2679 ...    1    0    0]]
[[  52    1  362 ...    0    1    0]
 [  30    2 3069 ...    1    0    0]
 [  34    3 6850 ...    1    0    0]
 ...
 [  68    2 6761 ...    0    0    1]
 [  28    3 2606 ...    1    0    0]
 [  29    3 2679 ...    1    0    0]]
[[  52    1  362 ...    0    1    0]
 [  30    2 3069 ...    1    0    0]
 [  34    3 6850 ...    1    0    0]
 ...
 [  68    2 6761 ...    0    0    1]
 [  28    3 2606 ...    1    0    0]
 [  29    3 2679 ...    1    0    0]]
[[  52    1  362 ...    0    1    0]
 [  30    2 3069 ...    1    0    0]
 [  34    3 68

 [  29    3 2679 ...    1    0    0]]
[[  52    1  362 ...    0    1    0]
 [  30    2 3069 ...    1    0    0]
 [  34    3 6850 ...    1    0    0]
 ...
 [  68    2 6761 ...    0    0    1]
 [  28    3 2606 ...    1    0    0]
 [  29    3 2679 ...    1    0    0]]
[[  52    1  362 ...    0    1    0]
 [  30    2 3069 ...    1    0    0]
 [  34    3 6850 ...    1    0    0]
 ...
 [  68    2 6761 ...    0    0    1]
 [  28    3 2606 ...    1    0    0]
 [  29    3 2679 ...    1    0    0]]
[[  52    1  362 ...    0    1    0]
 [  30    2 3069 ...    1    0    0]
 [  34    3 6850 ...    1    0    0]
 ...
 [  68    2 6761 ...    0    0    1]
 [  28    3 2606 ...    1    0    0]
 [  29    3 2679 ...    1    0    0]]
[[  52    1  362 ...    0    1    0]
 [  30    2 3069 ...    1    0    0]
 [  34    3 6850 ...    1    0    0]
 ...
 [  68    2 6761 ...    0    0    1]
 [  28    3 2606 ...    1    0    0]
 [  29    3 2679 ...    1    0    0]]
[[  52    1  362 ...    0    1    0]
 [  30    2 3

 [  29    3 2679 ...    1    0    0]]
[[  52    1  362 ...    0    1    0]
 [  30    2 3069 ...    1    0    0]
 [  34    3 6850 ...    1    0    0]
 ...
 [  68    2 6761 ...    0    0    1]
 [  28    3 2606 ...    1    0    0]
 [  29    3 2679 ...    1    0    0]]
[[  52    1  362 ...    0    1    0]
 [  30    2 3069 ...    1    0    0]
 [  34    3 6850 ...    1    0    0]
 ...
 [  68    2 6761 ...    0    0    1]
 [  28    3 2606 ...    1    0    0]
 [  29    3 2679 ...    1    0    0]]
[[  52    1  362 ...    0    1    0]
 [  30    2 3069 ...    1    0    0]
 [  34    3 6850 ...    1    0    0]
 ...
 [  68    2 6761 ...    0    0    1]
 [  28    3 2606 ...    1    0    0]
 [  29    3 2679 ...    1    0    0]]
[[  52    1  362 ...    0    1    0]
 [  30    2 3069 ...    1    0    0]
 [  34    3 6850 ...    1    0    0]
 ...
 [  68    2 6761 ...    0    0    1]
 [  28    3 2606 ...    1    0    0]
 [  29    3 2679 ...    1    0    0]]
[[  52    1  362 ...    0    1    0]
 [  30    2 3

In [ ]:
lr.predict([[23, 1, 3573, 12, 0, 0, 0, 0, 1, ]])

In [44]:
df1

,Age,Job,Credit amount,Duration,Purpose_car,Purpose_domestic appliances,Purpose_education,Purpose_furniture/equipment,Purpose_radio/TV,Purpose_repairs,...,Savings_no_inf,Savings_quite rich,Savings_rich,Risk_bad,Check_moderate,Check_no_inf,Check_rich,Age_cat_Young,Age_cat_Adult,Age_cat_Senior
480,23,1,3573,12,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
822,41,2,2712,36,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
29,63,2,6836,60,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
788,50,2,6224,48,0,0,1,0,0,0,...,0,0,0,1,1,0,0,0,1,0
307,33,1,727,12,0,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,22,2,3632,24,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
231,51,1,2507,9,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0
542,31,2,6350,30,0,0,0,1,0,0,...,1,0,0,1,0,0,0,1,0,0
792,27,2,1221,6,0,0,0,1,0,0,...,1,0,0,0,0,1,0,1,0,0


In [0]:
df1['prediction'] = y_pred.tolist()

In [0]:
org_df['predicted'] = y_pred.tolist()

In [0]:
import pickle

with open('/content/drive/My Drive/Analytics Projects | POC/Risk Analytics/trained_model.model','wb') as f:
    pickle.dump(fitted_bag, f)

In [39]:
#Logistics Regression
lr = LogisticRegression(C = 3.0, solver = 'liblinear', multi_class = 'auto')
lr.fit(x,y)
y_predict = lr.predict(x_test)
accuracy_score(y_test,y_predict)

0.785

In [0]:
#Random Forest 
rf_model = RandomForestClassifier(n_estimators= 2)
rf_model.fit(x,y)
test_pred = rf_model.predict(x_test)
accuracy_score(y_test, test_pred)

0.845

In [0]:
#Naive bayes
NB = GaussianNB()
nb_model = NB.fit(x,y)
y_predi = nb_model.predict(x_test)
accuracy_score(y_test,y_predi)

0.73

In [0]:
#Decision Tree
DT = DecisionTreeClassifier()
dt_model = DT.fit(x,y)
test_predict = dt_model.predict(x_test)
accuracy_score(y_test,test_predict)

1.0

In [0]:
#SVC 
clf = SVC(C=7, gamma = 'auto', kernel = 'linear', class_weight='balanced')
svc_trained_model = clf.fit(x,y)
y_predt = clf.predict(x_test)
accuracy_score(y_test, y_predt)